# IBA Probe for Specific Interface Flapping

In this tutorial we will demonstrate the "Specific Interface Flapping" probe that is used to messure and alert on specific interfaces that are flapping up/down.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [1]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our probe

In [2]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [3]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [4]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the interface flapping probe from the library

In [5]:
from aospy.ibaprobelib.specific_interface_flapping import specific_interface_flapping_probe

Next let's create the interface flapping probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [ ]:
?specific_interface_flapping_probe

The following paramaters are required:

`label` - Descriptive name for the probe  
`interfaces` - list of interfaces   
    - system_label - label of system node that hosts given interface  
    - interface_name - if_name of interface node representing given interface  
   
`threshold` - sum total of number of flaps in recent-history for which an anomaly will be raised  
`duration` - number of seconds of recent-history in which interface flapping will be considered  
`anomaly_history_count` - number of anomaly flaps that will be recorded for inspection  
`max_flapping_interfaces_percentage` - maximum percentage of flapping interfaces on a device  

So for the case of reuse, we will define these variables here:

In [6]:
probe_label = 'DC1-Spine1-flapping'
interfaces = [
            {'system_label':'spine1', 'interface_name':'Ethernet1'},
             {'system_label':'spine1', 'interface_name':'Ethernet2'}
             ]
threshold = 8
duration = 60
anomaly_history_count = 500
max_flapping_interfaces_percentage = 10

Notice how `interface` is used to provide a list of interfaces for use with the probe.  
To fined the interfaces for each node you can look in the Links section of the blueprint

<img src="media/specific_hotcold2.png">

Now let's create the body data from these variables:

In [7]:
probe_body = specific_interface_flapping_probe(label=probe_label, 
                                      interfaces=interfaces,  
                                      threshold=threshold, 
                                      duration=duration, 
                                      anomaly_history_count=anomaly_history_count, 
                                      max_flapping_interfaces_percentage=max_flapping_interfaces_percentage)

If you'd like to dump the contents of the probe_body, you can use the following:

In [8]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [9]:
probes_url = bp.url + "/probes"

In [10]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [11]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [12]:
resp.ok

True

Now we have our IBA Probe ID:

In [13]:
probe_id = resp.json()['id']

print probe_id

62ac5ea9-1b5e-40f4-847d-2778e1e5e076


And now we can see the IBA active probe in the UI:

<img src="media/specific_flap1.png">

And then select the probe on the UI to get the details:

<img src="media/specific_flap2.png">

# Delete the IBA Interface Flapping Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [14]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True